In [1]:
import numpy as np
import pandas as pd

## 데이터 준비

In [2]:
nsmc_train_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep='\t', encoding='utf8', engine='python')
nsmc_train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [3]:
nsmc_test_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep='\t', encoding='utf8', engine='python')
nsmc_test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


# 전처리 작업

In [4]:
nsmc_train_df=nsmc_train_df[nsmc_train_df['document'].notnull()]
nsmc_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [5]:
nsmc_train_df['label'].value_counts()  #label(0):부정, labe(1):긍정

,count
label,
0,75170
1,74825


In [6]:
#한글 아닌것 제외
import re
nsmc_train_df['document']=nsmc_train_df['document'].apply(lambda x:re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',x))
nsmc_train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [7]:
nsmc_test_df.info()   #document에 null값 3개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [8]:
#document컬럼ㄹ이 null인 샘플제거
nsmc_test_df=nsmc_test_df[nsmc_test_df['document'].notnull()]
nsmc_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [9]:
#타겟 컬럼 label확인(0:부정, 1:긍정)
print(nsmc_train_df['label'].value_counts())

label
0    75170
1    74825
Name: count, dtype: int64


# 모델 구축

In [11]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 33.4 MB/s eta 0:00:00


In [12]:
#형태소 분석해서 토큰화:한글형태로 엔진으로 Okt이용
from konlpy.tag import Okt
okt=Okt()

In [13]:
#문장을 토큰화하기위해 okt_tokenizer함수를 정의하고 okt.morphs()함수 사용해 형태소 단위로 토큰화 작업 수행
def okt_tokenizer(text):
  tokens=okt.morphs(text)
  return tokens

In [14]:
# TF-IDF 기반 피처 벡터 생성(중요! 실행시 약 20분 소요)
# 사이킷런의 TfidfVectorizer를 이용하여 TF-IDF 벡터화에 사용할 tfidf 객체를 생성
# 토큰 생성기(tokenizer)는 우리가 정의한 okt_tokenizer() 함수로 설정하고 토큰의 단어 크기(ngram_range)는 1~2개 단어로 함
# 토큰은 출현 빈도가 최소(min_df) 3번 이상이고 최대(max_df) 90% 이하인 것만 사용
# 벡터화할 데이터(nsmc_train_df['document']) 에 대해 벡터 모델 tfidf의 내부 설정값을 조정(fit())하고 벡터로 변환을 수행(transform())

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1, 2), min_df=3, max_df=0.9)
tfidf.fit(nsmc_train_df['document'])
nsmc_train_tfidf = tfidf.transform(nsmc_train_df['document'])

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [15]:
#로지스틱 회귀 기반 분석모델 생성
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=0)

#nsmc_train_tfidf를 독립변수 X로하고, label컬럼을 종속변수 Y로 하여 로지스틱 회귀모델 학습
model.fit(nsmc_train_tfidf, nsmc_train_df['label'])

LogisticRegression(random_state=0)

In [16]:
# 로지스틱 회귀의 best 하이퍼파라미터 찾기
# GridSearchCV: 모델의 성능향상을 위해 쓰이는 기법, 하이퍼파라미터 값에 대한 경우의 수마다 예측 성능을 측정 평가하여 비교
from sklearn.model_selection import GridSearchCV

# 하이퍼 매개변수 C에 대해 비교 검사를 할 6개 값[1,3,3.5,4,4.5,5]을 params로 하고 교차 검증cv를 3(fold 횟수)
# 모형 비교 기준은 정확도로 설정(scoring='accuracy')하여 GridSearchCV 객체를 생성
# verbose: 보여주는 형태 (1: 간단하게 보여주기)
params = {'C': [1, 3, 3.5, 4, 4.5, 5]}
SA_lr_grid_cv = GridSearchCV(model, param_grid=params, cv=3, scoring='accuracy', verbose=1)

In [17]:
SA_lr_grid_cv.fit(nsmc_train_tfidf, nsmc_train_df['label'])

Fitting 3 folds for each of 6 candidates, totalling 18 fits


GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1, 3, 3.5, 4, 4.5, 5]}, scoring='accuracy',
             verbose=1)

In [18]:
#GridSearchCV에 의해 찾은 최적의 C 매개변수(best_params)와 최고점수(best_score)을 출력하여 확인
print(SA_lr_grid_cv.best_params_, round(SA_lr_grid_cv.best_score_,4))

{'C': 4} 0.8551


In [19]:
#최적 파라미터의 best모델 저장
SA_lr_best=SA_lr_grid_cv.best_estimator_

In [20]:
import joblib
joblib.dump(tfidf, 'tfidf.pkl')
joblib.dump(SA_lr_best, 'SA_lr_best.pkl')

['SA_lr_best.pkl']

In [21]:
nsmc_test_tfidf=tfidf.transform(nsmc_test_df['document'])

In [22]:
test_predict=SA_lr_best.predict(nsmc_test_tfidf)

In [23]:
from sklearn.metrics import accuracy_score
  print('감성분석정확도:',round(accuracy_score(nsmc_test_df['label'],test_predict),3))

감성분석정확도: 0.858


In [29]:
#flask로 실행하고 싶을 때
import sklearn
print(sklearn.__version__)  #visualstudio cmd에서 pip show scikit-learn하고 버전확인
#pip install --upgrade scikit-learn==1.6.1 버전 맞춰줘야함

1.6.1


# 감성 예측

In [24]:
st=input("문장입력>>")

문장입력>>대박


In [25]:
#입력 테스트에 대한 전처리 수행
st2=re.compile(r'[ㄱ-ㅣ가-힣]+').findall(st)
print(st2)
st3=[''.join(st2)]
print(st3)

['대박']
['대박']


In [26]:
#입력 텍스트의 피처 벡터화
st_tfidf=tfidf.transform(st3)

In [27]:
#최적 감성분석 모델에 적용하여 감성 분석 평가
st_predict=SA_lr_best.predict(st_tfidf)
st_predict

array([1])

In [28]:
#예측값 출력하기
if(st_predict==0):
  print(st,'--> 부정 감성')
else:
  print(st, '--> 긍정 감성')

대박 --> 긍정 감성
